<a href="https://colab.research.google.com/github/ark2016/ML-TalentMatch/blob/main/ML_TalentMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install transformers

In [28]:
!pip install torch

In [29]:
!pip install evaluate

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [31]:
def extract_text_from_docx(path_to_file):
    with open(path_to_file, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
        texts = [item['content'] for item in data]
        #print(texts[0])
        return texts
    return 'error'

In [32]:
text = extract_text_from_docx('resume_texts_without_lowercase.json')

In [33]:
from transformers import pipeline

pipe = pipeline("question-answering", model="Kiet/autotrain-resume_parser-1159242747")

from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("Kiet/autotrain-resume_parser-1159242747")
model = AutoModelForQuestionAnswering.from_pretrained("Kiet/autotrain-resume_parser-1159242747")

Some weights of the model checkpoint at Kiet/autotrain-resume_parser-1159242747 were not used when initializing LongformerForQuestionAnswering: ['longformer.embeddings.position_ids']
- This IS expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at Kiet/autotrain-resume_parser-1159242747 were not used when initializing LongformerForQuestionAnswering: ['longformer.embeddings.position_ids']
- This IS expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model trained on another task or with an

In [19]:
answer = pipe( "What is my mail?", text[176])

Input ids are automatically padded from 357 to 512 to be a multiple of `config.attention_window`: 512


In [18]:
name = answer["answer"]

'Anton'

In [ ]:
answer

In [13]:
text[176]

'STEPAN\nE X P E R I E N C E\nGOLDOBIN\nUI/UX DESIGNER / FRONTEND |\n09.2019 - now\nClearscale (AWS Partner), USA\n27 years old\nRare unicorn. Working with: React, Angular, Ember, GraphQL, Node.js, Express. Also\nmaking UX/UI design and graphic design.\nUI/UX DESIGNER / FRONTEND | 05.2019 - 09-2019\nQIWI, Moscow\nDeveloped a service for companies, as a “Kickstarter”, but for banks/startups. Created\nUX/UI design and a frontend part (React).\nUI/UX DESIGNER / FRONTEND | 09.2018 - 05.2019\nUI/ UX Designer Evercity, Moscow\nI have developed UI/UX and prototypes for the web service.\nFrontend Developer\nCreating front end visualisation in React.js\nMotion Designer\nMade a more iterative development process with flexible course change.\nUI/UX DESIGNER / FRONTEND | 03.2017 - 09.2019\nMSTlab24, St.Petersburg\nMy name is Anton and I am a\nWith a lot of experience in SCRUM methodology, I develop in several spheres:\ncross-functional specialist with UI /\nDevelopment of UI/UX of mobile and web a

In [35]:
!pip install docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53895 sha256=dc927737ecf03608f13f40ad25c412bbe0c13866b7ae788355d1bc77ee4941b0
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx


In [40]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=184bede84b00dca7a295b7a7c8694eeb9f21b4942458cc3ac84734727f373413
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [41]:
import os

import nltk
# from docx import Document
# import pdfplumber
# import json
from nltk.corpus import wordnet
import pymorphy2



In [42]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [44]:


# def file_to_text(file_path):
#     try:
#         text = ""
#         if ".docx" in file_path:
#             # Преобразование .docx файла в текст
#             doc = Document(file_path)
#             for paragraph in doc.paragraphs:
#                 text += paragraph.text.replace("\t"," ").strip() + ' \n '
#         else:е
#             # Преобразование .pdf файла в текст
#             with pdfplumber.open(file_path) as pdf:
#                 for page in pdf.pages:
#                     text += page.extract_text() + ' \n '

#         #text = text.lower()
#         work_with_text(text)
#         return text
#     except Exception as e:
#         print(f"Ошибка при чтении файла: {e}")
#         return None

# def work_with_text(text):
#     if "akhundov damat" in text:
#         print(text)


def split_resume_into_blocks(text, keywords):
    # Получение индексов начала каждого ключевого слова
    start_indices = []
    for keyword, synonyms in keywords.items():
        for synonym in synonyms:
            start_index = text.lower().find(synonym.lower())
            if start_index != -1:
                start_indices.append((start_index, keyword))
                break

    # Сортировка индексов
    start_indices.sort()

    # Разделение текста на блоки по ключевым словам
    blocks = {}
    for i in range(len(start_indices)):
        start_index, block_name = start_indices[i]
        end_index = start_indices[i + 1][0] if i + 1 < len(start_indices) else len(text)
        blocks[block_name] = text[start_index:end_index].strip()

    return blocks

def extract_remaining_text(text, blocks):
    remaining_text = text
    for block_name, block_text in blocks.items():
        remaining_text = remaining_text.replace(block_text, "")
    return remaining_text.strip()

# Функция для получения синонимов с использованием nltk
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

# Функция для получения синонимов с использованием pymorphy2
def get_russian_synonyms(word):
    morph = pymorphy2.MorphAnalyzer()
    parsed_word = morph.parse(word)[0]
    return [parsed_word.normal_form] + [i.normal_form for i in parsed_word.lexeme]

# Список ключевых слов для блоков
keywords = {
    "education": ["education", "учеба"],
    "awards": ["awards", "награды"],
    "experience": ["experience", "опыт"],
    "projects": ["projects", "проекты"],
    "skills": ["skills", "навыки"],
    "activities": ["activities", "деятельность"]
}




# if __name__ == "__main__":
# Добавление синонимов к ключевым словам
for key, value in keywords.items():
    synonyms = []
    for word in value:
        synonyms.extend(get_synonyms(word))  # Для английских синонимов
        synonyms.extend(get_russian_synonyms(word))  # Для русских синонимов
    keywords[key] = list(set(synonyms))

print(keywords)

# file_path = "resume/AHMAT SULEIMENOV.docx"
# resume_text = file_to_text(file_path)
resume_text = text[0]

resume_blocks = split_resume_into_blocks(resume_text, keywords)
for i in resume_blocks.keys():
    print(i, ":", resume_blocks[i])
remaining_text = extract_remaining_text(resume_text, resume_blocks)
print("Remaining Text: ", remaining_text)
# with open('resume_texts_without_lowercase.json', 'w', encoding='utf-8') as json_file:
#     json.dump(resume_texts, json_file, ensure_ascii=False, indent=4)
#     print("JSON файл успешно сохранен.")

{'education': ['pedagogy', 'training', 'Education_Department', 'education', 'Department_of_Education', 'educational_activity', 'Education', 'didactics', 'учёба', 'instruction', 'breeding', 'teaching'], 'awards': ['grant', 'accolade', 'awards', 'award', 'honour', 'награда', 'prize', 'laurels', 'honor', 'present', 'awarding'], 'experience': ['go_through', 'experience', 'live', 'see', 'have', 'feel', 'get', 'receive', 'know', 'опыт'], 'projects': ['throw', 'send_off', 'contrive', 'propose', 'project', 'jut', 'plan', 'проект', 'cast', 'externalise', 'fancy', 'jut_out', 'projects', 'figure', 'protrude', 'see', 'image', 'undertaking', 'picture', 'externalize', 'task', 'visualise', 'labor', 'stick_out', 'visualize', 'design', 'projection', 'envision'], 'skills': ['attainment', 'accomplishment', 'science', 'acquirement', 'skills', 'acquisition', 'skill', 'навык'], 'activities': ['деятельность', 'activities', 'activity', 'natural_action', 'body_process', 'bodily_function', 'action', 'activeness

In [156]:
resume_blocks["experience"] +

'EXPERIENCE \n Lead  Developer Jun 2021 - Present \n Impact Admissions Nur-Sultan, KZ \n Programmed a bot to modify scores of 98 students and post leader board in telegram and Google Sheets. \n Researched market of 10 Ed Tech companies, compared their gains, and tested their platform on 20 students. \n Developing an admissions prediction platform that analyzes student’s essays, exams, and grades \n Unix  Lab  Tutor Aug 2021 - Present \n New York University Abu Dhabi Abu Dhabi, UAE \n Assisting students with their assignments, CS concepts, and personal tech'

In [49]:
pipe("education", text[0])

Input ids are automatically padded from 174 to 512 to be a multiple of `config.attention_window`: 512


{'score': 0.3767259418964386,
 'start': 134,
 'end': 167,
 'answer': 'New  York  University  Abu  Dhabi'}

In [50]:
resume_blocks["awards"]

'AWARDS \n Astana Innovations Challenge Hackathon - Top 2 among 50+ teams \n Republican Mathematics Olympiad Bronze Medalist - Top 15 among 200+ participants \n Future Leaders Exchange Program (FLEX) - Top 2% among 5000+ applicants \n SKILLS \n Programming Languages: Python, C++, C, Dart, JavaScript, HTML/CSS \n Frameworks/Tools: Git, Flutter, SQL, Pandas, Photoshop, Figma'

In [55]:
pipe("all awards", text[0])

{'score': 0.06279513984918594,
 'start': 2200,
 'end': 2264,
 'answer': '\n Astana Innovations Challenge Hackathon - Top 2 among 50+ teams'}

In [54]:
pipe("Programming Languages", text[0])

{'score': 0.3734247088432312,
 'start': 2455,
 'end': 2497,
 'answer': 'Python, C++, C, Dart, JavaScript, HTML/CSS'}

In [52]:
resume_blocks["experience"]

'EXPERIENCE \n Lead  Developer Jun 2021 - Present \n Impact Admissions Nur-Sultan, KZ \n Programmed a bot to modify scores of 98 students and post leader board in telegram and Google Sheets. \n Researched market of 10 Ed Tech companies, compared their gains, and tested their platform on 20 students. \n Developing an admissions prediction platform that analyzes student’s essays, exams, and grades \n Unix  Lab  Tutor Aug 2021 - Present \n New York University Abu Dhabi Abu Dhabi, UAE \n Assisting students with their assignments, CS concepts, and personal tech'

In [53]:
pipe("experience", text[0])

{'score': 0.08028198033571243,
 'start': 802,
 'end': 856,
 'answer': 'Lead  Developer Jun 2021 - Present \n Impact Admissions'}

In [56]:
resume_blocks["projects"]

'projects \n Leading CS related workshops on a monthly basis with more than 30 students attending regularly \n PROJECTS \n Fire Fighting Robot  Python/Gearsbot May 2021 - Jul 2021 \n Constructed a robot system able to detect survivors in burning buildings and clear obstacles in its path \n Virtual  File  System   C++/STL/OOP/Linux/Sorting algorithms Mar 2021 - Apr 2021 \n Implemented a user-friendly, private file system on C++ \n Designed the ability to modify file using Linux commands \n Doodle Jump   Python/Processing Nov 2020 - Dec 2020 \n Programmed from scratch a Doodle jump game on Processing IDE \n ACTIVITES \n Google Developers Club - Co-Lead, supervising work of various teams Aug 2021 - Present \n ACM ICPC Team - Team Leader, competing in ACM ICPC Jun 2021 - Present \n IEEE - Member, competing in IEEEXtreme and hackathons Dec 2020 - Present'

In [57]:
pipe("projects", text[0])

{'score': 0.004002835135906935,
 'start': 1460,
 'end': 1498,
 'answer': '\n Fire Fighting Robot  Python/Gearsbot'}

In [60]:
pipe("What language do I speak?", text[0])

{'score': 0.00015950892702676356,
 'start': 800,
 'end': 856,
 'answer': '\n Lead  Developer Jun 2021 - Present \n Impact Admissions'}

In [61]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=d4dcd725ad7cf716c4b8f307a11fc98c377c2b2050d971c07daead71e3ccf939
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [62]:
from langdetect import detect
detect("War doesn't show who's right, just who's left.")

'en'

In [64]:
detect(text[0])

'en'

In [66]:
dicta = {
  "resume": {
      "resume_id": "",
      "first_name": "",
      "last_name": "",
      "middle_name": "",
      "birth_date": "",
      "birth_date_year_only": "",
      "country": "",
      "city": "",
      "about": "",
      "key_skills": "",
      "salary_expectations_amount": "",
      "salary_expectations_currency": "",
      "photo_path": "",
      "gender": "",
      "language": "",
      "resume_name": "",
      "source_link": "",
      "contactItems": [
        {
          "resume_contact_item_id": "",
          "value": "",
          "comment": "",
          "contact_type": ""
        }
      ],
      "educationItems": [
        {
          "resume_education_item_id": "",
          "year": "",
          "organization": "",
          "faculty": "",
          "specialty": "",
          "result": "",
          "education_type": "",
          "education_level": ""
        }
      ],
      "experienceItems": [
        {
          "resume_experience_item_id": "",
          "starts": "",
          "ends": "",
          "employer": "",
          "city": "",
          "url": "",
          "position": "",
          "description": "",
          "order": ""
        }
      ],
      "languageItems": [
        {
          "resume_language_item_id": "",
          "language": "",
          "language_level": ""
        }
      ]
    }
}

In [84]:
dicta['resume']["resume_id"] = pipe("resume_id", resume_text)['answer']

In [75]:
FullName = pipe("what is my name", resume_text)['answer']

In [85]:
names = ["first_name", "last_name", "middle_name"]
i = 0
for name in FullName.split():
    dicta['resume'][names[i]] = name
    i += 1

In [87]:
birth_date = pipe("birth_date", resume_text)['answer']

In [88]:
dicta['resume']["birth_date"] = birth_date

In [90]:
dicta['resume']["birth_date_year_only"] = (birth_date==True)

In [99]:
homeland = pipe("homeland", resume_text)['answer']

In [186]:
dicta['resume']["country"] = homeland

In [96]:
hometown = pipe("hometown", resume_text)['answer']

In [97]:
dicta['resume']["city"] = hometown

In [102]:
about = pipe("describe me", resume_text)['answer']

In [103]:
dicta['resume']["about"] = about

In [105]:
key_skills = pipe("tell me key skills", resume_text)['answer']

In [106]:
dicta['resume']["key_skills"] = key_skills

In [109]:
dicta['resume']["language"] = detect(resume_text)

In [122]:
organization = pipe("place of education", resume_text)['answer']

In [126]:
dicta['resume']['educationItems'][0]["organization"] = organization

In [134]:
faculty = pipe("faculty of education", resume_text)['answer']

In [135]:
dicta['resume']['educationItems'][0]["faculty"] = faculty

In [140]:
specialty = pipe("specialty of education", resume_text)['answer']

In [141]:
dicta['resume']['educationItems'][0]["specialty"] = specialty

In [142]:
result = pipe("result of education", resume_text)['answer']

In [143]:
dicta['resume']['educationItems'][0]["result"] = result

In [145]:
education_type = pipe("type of education", resume_text)['answer']

In [146]:
dicta['resume']['educationItems'][0]["education_type"] = education_type

In [147]:
education_level = pipe("level of education", resume_text)['answer']

In [157]:
resume_blocks["experience"]

'EXPERIENCE \n Lead  Developer Jun 2021 - Present \n Impact Admissions Nur-Sultan, KZ \n Programmed a bot to modify scores of 98 students and post leader board in telegram and Google Sheets. \n Researched market of 10 Ed Tech companies, compared their gains, and tested their platform on 20 students. \n Developing an admissions prediction platform that analyzes student’s essays, exams, and grades \n Unix  Lab  Tutor Aug 2021 - Present \n New York University Abu Dhabi Abu Dhabi, UAE \n Assisting students with their assignments, CS concepts, and personal tech'

In [172]:
starts = pipe("start of work", resume_text )['answer']

In [173]:
dicta['resume']['experienceItems'][0]["starts"] = starts

In [174]:
ends = pipe("end of work", resume_text )['answer']

In [175]:
dicta['resume']['experienceItems'][0]["ends"] = ends

In [176]:
employer = pipe("place of work", resume_text )['answer']

In [177]:
dicta['resume']['experienceItems'][0]["employer"] = employer

In [179]:
city = pipe("city of work", resume_text )['answer']

In [180]:
dicta['resume']['experienceItems'][0]["city"] = city

In [181]:
position = pipe("position at work", resume_text )['answer']

In [182]:
dicta['resume']['experienceItems'][0]["position"] = position

In [187]:
dicta

{'resume': {'resume_id': 'Lead  Developer Jun 2021 - Present \n Impact Admissions',
  'first_name': 'AHMAT',
  'last_name': 'SULEIMENOV',
  'middle_name': '',
  'birth_date': 'May 2024',
  'birth_date_year_only': False,
  'country': 'New  York  University  Abu  Dhabi',
  'city': 'New  York  University  Abu  Dhabi',
  'about': '\n Lead  Developer Jun 2021 - Present \n Impact Admissions',
  'key_skills': 'Python, C++, C, Dart, JavaScript, HTML/CSS',
  'salary_expectations_amount': '',
  'salary_expectations_currency': '',
  'photo_path': '',
  'gender': '',
  'language': 'en',
  'resume_name': '',
  'source_link': '',
  'contactItems': {'resume_contact_item_id': '',
   'value': '+971 50 000 9004 ⬦',
   'comment': '',
   'contact_type': ''},
  'educationItems': [{'resume_education_item_id': '',
    'year': '',
    'organization': 'New  York  University  Abu  Dhabi',
    'faculty': 'New York University Abu Dhabi Abu Dhabi, UAE',
    'specialty': 'B.S. of Computer Science and Mathematics',
